# Βελτιστοποίηση Μοντέλων Open AI

Αυτό το notebook βασίζεται στις τρέχουσες οδηγίες που παρέχονται στην τεκμηρίωση [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) της Open AI.

Η βελτιστοποίηση βελτιώνει την απόδοση των βασικών μοντέλων για την εφαρμογή σας, επανεκπαιδεύοντάς τα με επιπλέον δεδομένα και πλαίσιο που σχετίζονται με τη συγκεκριμένη περίπτωση χρήσης ή σενάριο. Σημειώστε ότι τεχνικές όπως το _few shot learning_ και το _retrieval augmented generation_ σας επιτρέπουν να ενισχύσετε το προεπιλεγμένο prompt με σχετικά δεδομένα για καλύτερη ποιότητα. Ωστόσο, αυτές οι προσεγγίσεις περιορίζονται από το μέγιστο μέγεθος παραθύρου tokens του εκάστοτε βασικού μοντέλου.

Με τη βελτιστοποίηση, ουσιαστικά επανεκπαιδεύουμε το ίδιο το μοντέλο με τα απαραίτητα δεδομένα (επιτρέποντάς μας να χρησιμοποιήσουμε πολύ περισσότερα παραδείγματα από όσα χωράνε στο μέγιστο παράθυρο tokens) - και αναπτύσσουμε μια _προσαρμοσμένη_ έκδοση του μοντέλου που πλέον δεν χρειάζεται να του παρέχουμε παραδείγματα κατά το inference. Αυτό όχι μόνο βελτιώνει την αποτελεσματικότητα του σχεδιασμού των prompts μας (έχουμε μεγαλύτερη ευελιξία στη χρήση του παραθύρου tokens για άλλα πράγματα), αλλά ενδεχομένως μειώνει και το κόστος μας (καθώς μειώνεται ο αριθμός των tokens που χρειάζεται να στείλουμε στο μοντέλο κατά το inference).

Η βελτιστοποίηση περιλαμβάνει 4 βήματα:
1. Προετοιμάστε και ανεβάστε τα δεδομένα εκπαίδευσης.
1. Εκτελέστε τη διαδικασία εκπαίδευσης για να αποκτήσετε ένα βελτιστοποιημένο μοντέλο.
1. Αξιολογήστε το βελτιστοποιημένο μοντέλο και επαναλάβετε για καλύτερη ποιότητα.
1. Αναπτύξτε το βελτιστοποιημένο μοντέλο για inference όταν είστε ικανοποιημένοι.

Σημειώστε ότι δεν υποστηρίζουν όλα τα βασικά μοντέλα τη βελτιστοποίηση - [ελέγξτε την τεκμηρίωση της OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) για τις πιο πρόσφατες πληροφορίες. Μπορείτε επίσης να βελτιστοποιήσετε ένα μοντέλο που έχει ήδη βελτιστοποιηθεί. Σε αυτό το tutorial, θα χρησιμοποιήσουμε το `gpt-35-turbo` ως το βασικό μοντέλο στόχο για βελτιστοποίηση.

---


### Βήμα 1.1: Προετοιμάστε το Dataset σας

Ας δημιουργήσουμε ένα chatbot που θα σας βοηθά να κατανοήσετε τον περιοδικό πίνακα των στοιχείων, απαντώντας σε ερωτήσεις για ένα στοιχείο με ένα λιμερικό. Σε _αυτό_ το απλό tutorial, θα φτιάξουμε απλώς ένα dataset για να εκπαιδεύσουμε το μοντέλο με μερικά παραδείγματα απαντήσεων που δείχνουν τη μορφή που περιμένουμε να έχει τα δεδομένα. Σε μια πραγματική εφαρμογή, θα χρειαστεί να δημιουργήσετε ένα dataset με πολύ περισσότερα παραδείγματα. Ίσως να μπορείτε επίσης να χρησιμοποιήσετε ένα ανοιχτό dataset (για τον τομέα της εφαρμογής σας) αν υπάρχει, και να το προσαρμόσετε για χρήση στη διαδικασία fine-tuning.

Επειδή εστιάζουμε στο `gpt-35-turbo` και θέλουμε μια απάντηση μίας στροφής (chat completion), μπορούμε να δημιουργήσουμε παραδείγματα χρησιμοποιώντας [αυτή τη προτεινόμενη μορφή](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) που αντανακλά τις απαιτήσεις του OpenAI chat completion. Αν περιμένετε συνομιλίες με πολλές στροφές, θα χρησιμοποιούσατε τη [μορφή παραδείγματος πολλαπλών στροφών](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) που περιλαμβάνει μια παράμετρο `weight` για να δηλώσετε ποια μηνύματα θα χρησιμοποιηθούν (ή όχι) στη διαδικασία fine-tuning.

Θα χρησιμοποιήσουμε τη πιο απλή μορφή μίας στροφής για το tutorial μας εδώ. Τα δεδομένα είναι στη [μορφή jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) με 1 εγγραφή ανά γραμμή, κάθε μία ως αντικείμενο σε μορφή JSON. Το παρακάτω απόσπασμα δείχνει 2 εγγραφές ως δείγμα – δείτε το [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) για το πλήρες δείγμα (10 παραδείγματα) που θα χρησιμοποιήσουμε στο tutorial fine-tuning. **Σημείωση:** Κάθε εγγραφή _πρέπει_ να ορίζεται σε μία μόνο γραμμή (και όχι σε πολλές γραμμές όπως συνήθως σε ένα μορφοποιημένο αρχείο JSON)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Σε μια πραγματική εφαρμογή θα χρειαστείτε πολύ μεγαλύτερο σύνολο παραδειγμάτων για καλά αποτελέσματα – το δίλημμα είναι ανάμεσα στην ποιότητα των απαντήσεων και τον χρόνο/κόστος του fine-tuning. Εμείς χρησιμοποιούμε ένα μικρό σύνολο ώστε να ολοκληρώσουμε γρήγορα το fine-tuning και να δείξουμε τη διαδικασία. Δείτε [αυτό το παράδειγμα από το OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) για ένα πιο σύνθετο tutorial fine-tuning.


### Βήμα 1.2 Μεταφόρτωση του Συνόλου Δεδομένων σας

Μεταφορτώστε τα δεδομένα χρησιμοποιώντας το Files API [όπως περιγράφεται εδώ](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Σημειώστε ότι για να εκτελέσετε αυτόν τον κώδικα, πρέπει πρώτα να έχετε κάνει τα εξής:
 - Έχετε εγκαταστήσει το πακέτο Python `openai` (βεβαιωθείτε ότι χρησιμοποιείτε έκδοση >=0.28.0 για τις πιο πρόσφατες δυνατότητες)
 - Έχετε ορίσει τη μεταβλητή περιβάλλοντος `OPENAI_API_KEY` με το κλειδί API του OpenAI
Για περισσότερες πληροφορίες, δείτε τον [Οδηγό Εγκατάστασης](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) που παρέχεται για το μάθημα.

Τώρα, εκτελέστε τον κώδικα για να δημιουργήσετε ένα αρχείο προς μεταφόρτωση από το τοπικό σας αρχείο JSONL.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Βήμα 2.1: Δημιουργήστε την εργασία Fine-tuning με το SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Βήμα 2.2: Έλεγχος της Κατάστασης της εργασίας

Ακολουθούν μερικά πράγματα που μπορείτε να κάνετε με το API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Εμφανίζει τις τελευταίες n εργασίες fine-tuning
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Παίρνει λεπτομέρειες για μια συγκεκριμένη εργασία fine-tuning
- `client.fine_tuning.jobs.cancel(<job_id>)` - Ακυρώνει μια εργασία fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Εμφανίζει έως n συμβάντα από την εργασία
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Το πρώτο βήμα της διαδικασίας είναι η _επαλήθευση του αρχείου εκπαίδευσης_ για να βεβαιωθείτε ότι τα δεδομένα είναι στη σωστή μορφή.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Βήμα 2.3: Παρακολούθηση συμβάντων για την παρακολούθηση της προόδου


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Βήμα 2.4: Δείτε την κατάσταση στον Πίνακα Ελέγχου του OpenAI


Μπορείτε επίσης να δείτε την κατάσταση επισκεπτόμενοι την ιστοσελίδα της OpenAI και εξερευνώντας την ενότητα _Fine-tuning_ της πλατφόρμας. Εκεί θα δείτε την κατάσταση της τρέχουσας εργασίας, καθώς και να παρακολουθήσετε το ιστορικό προηγούμενων εκτελέσεων. Σε αυτό το στιγμιότυπο οθόνης, φαίνεται ότι η προηγούμενη εκτέλεση απέτυχε, ενώ η δεύτερη ολοκληρώθηκε με επιτυχία. Για να καταλάβετε τι συνέβη, η πρώτη εκτέλεση χρησιμοποίησε ένα αρχείο JSON με λανθασμένη μορφοποίηση εγγραφών – μόλις διορθώθηκε, η δεύτερη εκτέλεση ολοκληρώθηκε επιτυχώς και το μοντέλο έγινε διαθέσιμο για χρήση.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.el.png)


Μπορείτε επίσης να δείτε τα μηνύματα κατάστασης και τις μετρήσεις κάνοντας κύλιση πιο κάτω στον οπτικό πίνακα ελέγχου όπως φαίνεται:

| Μηνύματα | Μετρήσεις |
|:---|:---|
| ![Μηνύματα](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.el.png) |  ![Μετρήσεις](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.el.png)|


### Βήμα 3.1: Ανάκτηση ID & Δοκιμή του Fine-Tuned Μοντέλου στον Κώδικα


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Βήμα 3.2: Φόρτωση & Δοκιμή του Fine-Tuned Μοντέλου στο Playground

Τώρα μπορείτε να δοκιμάσετε το fine-tuned μοντέλο με δύο τρόπους. Πρώτα, μπορείτε να επισκεφθείτε το Playground και να χρησιμοποιήσετε το drop-down των Μοντέλων για να επιλέξετε το νέο σας fine-tuned μοντέλο από τις διαθέσιμες επιλογές. Η άλλη επιλογή είναι να χρησιμοποιήσετε το κουμπί "Playground" που εμφανίζεται στο πάνελ Fine-tuning (δείτε το παραπάνω screenshot), το οποίο ανοίγει την παρακάτω _συγκριτική_ προβολή που δείχνει δίπλα-δίπλα την έκδοση του foundation και του fine-tuned μοντέλου για γρήγορη αξιολόγηση.

Απλά συμπληρώστε το system context που χρησιμοποιήσατε στα δεδομένα εκπαίδευσης και δώστε την ερώτηση δοκιμής σας. Θα παρατηρήσετε ότι και οι δύο πλευρές ενημερώνονται με το ίδιο context και ερώτηση. Εκτελέστε τη σύγκριση και θα δείτε τη διαφορά στα αποτελέσματα μεταξύ τους. _Σημειώστε πώς το fine-tuned μοντέλο διαμορφώνει την απάντηση στη μορφή που δώσατε στα παραδείγματά σας, ενώ το foundation μοντέλο απλώς ακολουθεί το system prompt_.

Θα παρατηρήσετε επίσης ότι η σύγκριση εμφανίζει και τον αριθμό των tokens για κάθε μοντέλο, καθώς και τον χρόνο που χρειάστηκε για το inference. **Αυτό το συγκεκριμένο παράδειγμα είναι απλοϊκό και έχει σκοπό να δείξει τη διαδικασία, αλλά δεν αντικατοπτρίζει πραγματικό dataset ή σενάριο**. Ίσως παρατηρήσετε ότι και τα δύο δείγματα έχουν τον ίδιο αριθμό tokens (το system context και το user prompt είναι ίδια), με το fine-tuned μοντέλο να χρειάζεται περισσότερο χρόνο για inference (custom μοντέλο).

Σε πραγματικά σενάρια, δεν θα χρησιμοποιείτε ένα τόσο απλό παράδειγμα, αλλά θα κάνετε fine-tuning με πραγματικά δεδομένα (π.χ. κατάλογο προϊόντων για εξυπηρέτηση πελατών), όπου η διαφορά στην ποιότητα της απάντησης θα είναι πολύ πιο εμφανής. Σε _αυτό_ το πλαίσιο, για να πετύχετε αντίστοιχη ποιότητα απάντησης με το foundation μοντέλο θα χρειαστείτε περισσότερη custom prompt engineering, κάτι που θα αυξήσει τη χρήση tokens και πιθανώς τον σχετικό χρόνο επεξεργασίας για inference. _Για να το δοκιμάσετε, δείτε τα παραδείγματα fine-tuning στο OpenAI Cookbook για να ξεκινήσετε._



---

**Αποποίηση Ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να γνωρίζετε ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρανοήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
